In [1]:
import os
import shutil
import pandas as pd
from seeq import spy

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 196

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.workbooks

The `spy.workbooks` module provides functions for importing and exporting _workbooks_. A workbook is either a _Workbench Analysis_ (colored green in the Seeq user interface) or an _Organizer Topic_ (colored blue).

This functionality is helpful to:

- Move content between two Seeq servers
- Manage content by exporting and committing to a version control system like Git

The process typically starts by searching for some content that you have created in Seeq and exporting it. However, since this documentation needs some pre-built content to illustrate how it works, there is a pre-built-and-exported set of workbooks alongside this documentation notebook. So we'll go in a non-typical order of operations in this example.

## The Export Format

When content is exported from Seeq, each workbook is encapsulated in its own folder, including all its worksheets, calculated item definitions and all dependencies, journal/document images and anything else that is necessary to import that data into another server. Content is written to disk as either JSON, HTML or image files as appropriate. References to datasource items are also catalogued during export and default _datasource maps_ are created that facilitate identification of equivalent signals/conditions/scalars on the destination system so that the imported content gets "hooked up" to the right data.

## Main Actions

There are five main operations you can perform on workbooks:

- **search** for workbooks whose content you want to *pull*
- **pull** those workbooks into `Workbook` in-memory Python objects
- **save** the `Workbook` Python objects to disk in the export format described above
- **load** `Workbook` Python objects from disk into memory
- **push** in-memory `Workbook` Python objects into a Seeq Server

As mentioned, we're going to go out-of-order for illustration purposes: _load_, _push_, _search_, _pull_, _save_.

### Importing

This set of documentation comes with an _Example Export_ folder that contains an Analysis and a Topic for illustration purposes. First we **load** it into memory:

In [3]:
workbooks = spy.workbooks.load('Support Files/Example Export.zip')
workbooks

[Workbook "Users >> Agent API Key >> SPy Documentation Examples >> My Import >> Example Analysis" (0EFEEC61-3067-73E0-AD1A-BF8C12ECD440),
 Workbook "Users >> Agent API Key >> SPy Documentation Examples >> My Import >> Example Topic" (0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B)]

Now that the workbook definitions are in memory, we can push them into Seeq.

In [4]:
spy.workbooks.push(workbooks, path='SPy Documentation Examples >> My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,60,00:00:30.68,0,Success,0EFF30AD-83F3-FBE0-AD14-6C8C9BCD6CA9,link
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,00:00:14.32,0,Success,0EFF30AE-A69F-75D0-A572-688E64329931,link


,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,60,0:00:30.680972,0,Success,0EFF30AD-83F3-FBE0-AD14-6C8C9BCD6CA9,http://localhost:34216/0EFF30AD-8003-7710-86DB...
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,0:00:14.323748,0,Success,0EFF30AE-A69F-75D0-A572-688E64329931,http://localhost:34216/0EFF30AD-8003-7710-86DB...


The workbooks have been imported into Seeq in a _My Import_ folder with you as the owner. Refresh Seeq Workbench in your browser and take a look.

### Exporting

In Seeq Workbench, try changing the name of the _Example Analysis_ workbook to something like _My First Analysis Export_ so that you can tell that your changes get exported.

Now we will **search** for the workbooks we want to export. The syntax for a workbook search query is very similar to an item metadata search via `spy.search()`:

In [5]:
workbooks_df = spy.workbooks.search({
    'Path': 'SPy Documentation Examples >> My Import'
})

workbooks_df

,Time,Count
Results,00:00:00.26,2


,Archived,Created At,Creator ID,Creator Name,Creator Username,ID,Name,Owner ID,Owner Name,Owner Username,Path,Pinned,Search Folder ID,Type,Updated At,Workbook Type
0,NaN,2025-02-19 13:33:16.190595+00:00,0EFEEC4B-F5B5-E810-A713-FD55987D85F1,Agent API Key,agent_api_key,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,0EFEEC4B-F5B5-E810-A713-FD55987D85F1,Agent API Key,agent_api_key,SPy Documentation Examples >> My Import,NaN,0EFF2C6D-1E8D-EC40-84C8-E4815E4A172E,Workbook,2025-02-24 15:58:10.801380800+00:00,Analysis
1,NaN,2025-02-19 13:33:47.119516400+00:00,0EFEEC4B-F5B5-E810-A713-FD55987D85F1,Agent API Key,agent_api_key,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,0EFEEC4B-F5B5-E810-A713-FD55987D85F1,Agent API Key,agent_api_key,SPy Documentation Examples >> My Import,NaN,0EFF2C6D-1E8D-EC40-84C8-E4815E4A172E,Workbook,2025-02-19 13:36:51.077312700+00:00,Topic


As you can see, the `spy.workbooks.search()` command returns a metadata DataFrame with the properties of the workbooks. We can now use that to **pull**:

In [6]:
workbooks = spy.workbooks.pull(workbooks_df)
workbooks

,ID,Path,Name,Workbook Type,Count,Time,Errors,Result
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,SPy Documentation Examples >> My Import,Example Analysis,Analysis,0,00:00:02.23,0,Success
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,SPy Documentation Examples >> My Import,Example Topic,Topic,0,00:00:01.32,0,Success


[Workbook "My Folder >> SPy Documentation Examples >> My Import >> Example Topic" (0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B),
 Workbook "My Folder >> SPy Documentation Examples >> My Import >> Example Analysis" (0EFEEC61-3067-73E0-AD1A-BF8C12ECD440)]

These are the same type of in-memory Python objects that we had when we executed `spy.workbooks.load()`. Now we can **save** them to disk:

In [7]:
if os.path.exists('../My First Export'):
    shutil.rmtree('../My First Export')
    
spy.workbooks.save(workbooks, '../My First Export')

In the parent folder of this documentation notebook, you'll find a new _My First Export_ folder that contains similar files to the _Example Export_ folder that's part of the documentation.

## Inspecting Worksheets

With the in-memory Python objects that result from `spy.workbooks.pull()` or `spy.workbooks.load()`, you can inspect the worksheets to see what is displayed on them. For example, let's look at what's in the Details Pane of the second worksheet of _Example Analysis_:

In [8]:
worksheet_items = workbooks['Example Analysis'].worksheets['Calculated Items'].display_items
worksheet_items

,Name,ID,Type,Color,Line Style,Line Width,Lane,Samples Display,Axis Auto Scale,Axis Align,Axis Group,Axis Show,Selected
0,Area A_Temperature,0EFEEC52-EE9D-E8A0-A80B-A9863E02B47F,Signal,#068C45,Solid,1.0,3.0,Line,True,Left,A,True,False
1,Smooth Temperature,0EFEEC61-4DC6-EC60-854C-DC3DFC71BC70,Signal,#9D248F,Solid,2.5,3.0,Line,True,Left,A,True,False
2,Area A_Compressor Power,0EFEEC53-0FBC-E8B0-BC8C-ED8016F187BE,Signal,#CE561B,Solid,1.0,2.0,Line,True,Left,C,True,False
3,Area C_Temperature,0EFEEC53-0EB2-66E0-975C-8768328519FA,Signal,#00A2DD,Solid,1.0,4.0,Line,True,Left,D,True,False
4,High Power,0EFEEC61-5266-FDB0-99CB-A3724A00A719,Condition,#4055A3,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,False
5,Temperature Limit,0EFEEC61-4F12-ECE0-AA42-3D2B327D01DE,Scalar,#E1498E,Dash,1.0,3.0,NaN,True,Left,A,True,False


Now you can call `spy.pull()` to pull data for the items in the worksheet.

In [9]:
spy.pull(worksheet_items, start='2019-01-01T00:00:00', end='2019-01-02T00:00:00')

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EFEEC52-EE9D-E8A0-A80B-A9863E02B47F,Signal,Area A_Temperature,00:00:00.45,97,1,11 KB,Success
1,0EFEEC61-4DC6-EC60-854C-DC3DFC71BC70,Signal,Smooth Temperature,00:00:00.26,97,1,11 KB,Success
2,0EFEEC53-0FBC-E8B0-BC8C-ED8016F187BE,Signal,Area A_Compressor Power,00:00:00.21,97,1,11 KB,Success
3,0EFEEC53-0EB2-66E0-975C-8768328519FA,Signal,Area C_Temperature,00:00:00.32,97,1,11 KB,Success
4,0EFEEC61-5266-FDB0-99CB-A3724A00A719,Condition,High Power,00:00:00.21,1,1,64 B,Success
5,0EFEEC61-4F12-ECE0-AA42-3D2B327D01DE,Scalar,Temperature Limit,00:00:00.04,1,0,0 B,Success


,Area A_Temperature,Smooth Temperature,Area A_Compressor Power,Area C_Temperature,High Power,Temperature Limit
2019-01-01 00:00:00+00:00,93.861500,95.094408,39.189300,102.017778,1.0,120.433655
2019-01-01 00:15:00+00:00,95.364050,95.257065,39.842881,89.474636,1.0,120.433655
2019-01-01 00:30:00+00:00,95.056341,95.548184,39.949065,87.157081,1.0,120.433655
2019-01-01 00:45:00+00:00,96.938340,96.005686,39.424393,83.497438,1.0,120.433655
2019-01-01 01:00:00+00:00,97.057959,96.521943,39.326496,82.098203,1.0,120.433655
...,...,...,...,...,...,...
2019-01-01 23:00:00+00:00,72.263661,71.274814,0.002923,87.940288,0.0,120.433655
2019-01-01 23:15:00+00:00,72.065032,71.352077,0.002923,88.873322,0.0,120.433655
2019-01-01 23:30:00+00:00,71.991097,71.814958,0.002923,90.710400,0.0,120.433655
2019-01-01 23:45:00+00:00,72.166508,72.622069,0.002923,89.150583,0.0,120.433655


Note that if you just wanted the full metadata for the items, you could execute `spy.search(worksheet_items[['ID']])`.

## Manipulating Worksheets

In the example above, you saw how we can inspect the items that are displayed on the worksheet by accessing the `display_items` attribute on the worksheet object, which is a Pandas DataFrame.

You can make changes by manipulating this DataFrame as shown below. Documentation on the various attributes of a worksheet is available in the [spy.workbooks.AnalysisWorksheet](https://python-docs.seeq.com/reference/submodules/workbooks.html#seeq.spy.workbooks.AnalysisWorksheet) online documentation.

In [10]:
workbooks = spy.workbooks.load('Support Files/Example Export.zip')

worksheet = workbooks['Example Analysis'].worksheets['Details Pane']

# Copy this into a variable where each row is indexed by Name for easy manipulation
display_items = worksheet.display_items.set_index('Name')

# Change Area A_Optimizer color to red
display_items.at['Area A_Optimizer', 'Color'] = '#FF0000'

# Change Area A_Temperature line style
display_items.at['Area A_Temperature', 'Line Style'] = 'Short Dash'

# Remove Area A_Compressor Stage
display_items = display_items[~(display_items.index == 'Area A_Compressor Stage')]

# Add Area E_Temperature
area_e_temperature = spy.search({'Datasource Name': 'Example Data', 'Name': 'Area E_Temperature'}).set_index('Name')
area_e_temperature.at['Area E_Temperature', 'Color'] = '#00FF00'
area_e_temperature.at['Area E_Temperature', 'Lane'] = 2
area_e_temperature.at['Area E_Temperature', 'Selected'] = True
display_items = pd.concat([display_items, area_e_temperature])

# Reset the index since the worksheet expects a Name column, then assign the manipulated DataFrame to the worksheet
worksheet.display_items = display_items.reset_index()
worksheet.display_items

,Datasource Name,Name,Time,Count,Pages,Result
0,Example Data,Area E_Temperature,00:00:00.05,1,1,Success


,Name,ID,Type,Color,Line Style,Line Width,Lane,Samples Display,Axis Auto Scale,Axis Align,Axis Group,Axis Show,Selected,Value Unit Of Measure,Datasource Name,Archived
0,Area A_Temperature,0EFEEC52-EE9D-E8A0-A80B-A9863E02B47F,Signal,#E1498E,Short Dash,1.0,1.0,Line,True,Left,A,True,False,NaN,NaN,NaN
1,Area A_Optimizer,0EFEEC52-EFBB-62F0-814E-28FCBB29C7EA,Signal,#FF0000,Solid,1.0,2.0,Line,True,Left,B,True,False,NaN,NaN,NaN
2,Area E_Temperature,0EFEEC53-0FED-75F0-8495-8B09BE930F08,Signal,#00FF00,NaN,NaN,2.0,NaN,NaN,NaN,D,NaN,True,°F,Example Data,False


Now push to the server and click the link to observe:

In [11]:
spy.workbooks.push(workbooks, path='SPy Documentation Examples >> My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,61,00:00:19.60,0,Success,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,link
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,00:00:08.87,0,Success,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,link


,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,61,0:00:19.596672,0,Success,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,http://localhost:34216/0EFF2C6D-1E8D-EC40-84C8...
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,0:00:08.872527,0,Success,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,http://localhost:34216/0EFF2C6D-1E8D-EC40-84C8...


## Worksheet Display Configurations
### Trend View Configurations

You can access the trend toolbar configuration of a worksheet by accessing the `trend_toolbar` attribute on the worksheet object.

You can make changes by setting the respective properties.

In [8]:
workbooks = spy.workbooks.load('Support Files/Example Export.zip')

worksheet = workbooks['Example Analysis'].worksheets['Trend Toolbar']

# Turn off the Gridlines
worksheet.trend_toolbar.show_grid_lines = False

# Turn on the Dimming
worksheet.trend_toolbar.dimming = True

# Turn on the Certainity
worksheet.trend_toolbar.hide_uncertainty = True

worksheet.trend_toolbar

Trend toolbar configurations:
  - View: Trend
  - Show Gridlines: False
  - Hide Uncertainty: True
  - Dimming: True
  - Labels:
    - Signals:
      - Name: lane
      - Description: off
      - Asset: off
      - Asset Path Levels: 1
      - Line Style: off
      - Unit of Measure: off
      - Custom: off
      - Custom Labels: []
    - Conditions:
      - Name: off
      - Capsules: ['startTime', 'properties.Running']
    - Cursors:
      - Values: show
  - Color:
    - TrendItemsColor:
      - Name: Area A_Temperature, Color: #068C45
      - Name: Smooth Temperature, Color: #9D248F
      - Name: Area A_Compressor Power, Color: #CE561B
      - Name: Area C_Temperature, Color: #00A2DD
      - Name: Area A_Compressor Stage, Color: #AC4625
      - Name: High Power, Color: #4055A3
      - Name: Condition with Properties, Color: #4B7ABD
      - Name: Temperature Limit, Color: #E1498E
    - Capsules:
      - Color capsules by: capsuleProperty
      - Capsule property: Running
      - Caps

You can access the labels configuration of the trend toolbar by accessing the `labels` attribute on the `trend_toolbar` object.

You can make changes by setting the respective properties.

In [13]:
# Change Signals Name visibility
worksheet.trend_toolbar.labels.signals.name = 'lane'

# Change Signals Description visibility
worksheet.trend_toolbar.labels.signals.description = 'off'

# Change Signals Asset visibility
worksheet.trend_toolbar.labels.signals.asset = 'off'

# Change Signals Asset Path Level visibility
worksheet.trend_toolbar.labels.signals.asset_path_levels = 3

# Change Signals Line Style visibility
worksheet.trend_toolbar.labels.signals.line_style = 'off'

# Change Signals Unit of Measure visibility
worksheet.trend_toolbar.labels.signals.unit_of_measure = 'axis'

# Add Custom Labels to lanes
worksheet.trend_toolbar.labels.signals.custom = 'lane'
worksheet.trend_toolbar.labels.signals.custom_labels = ['Model TMP-3100Chill', 'Model CP-6400Delta']

# Change Conditions Name visibility and enable Capsules visibility
worksheet.trend_toolbar.labels.conditions.name = 'lane'
worksheet.trend_toolbar.labels.conditions.capsules = ['startTime', 'endTime', 'duration']

# Change Cursors Values visibility
worksheet.trend_toolbar.labels.cursors.values = 'show'

worksheet.trend_toolbar

Trend toolbar configurations:
  - View: Trend
  - Show Gridlines: False
  - Hide Uncertainty: True
  - Dimming: True
  - Labels:
    - Signals:
      - Name: lane
      - Description: off
      - Asset: off
      - Asset Path Levels: 3
      - Line Style: off
      - Unit of Measure: axis
      - Custom: lane
      - Custom Labels: ['Model TMP-3100Chill', 'Model CP-6400Delta']
    - Conditions:
      - Name: lane
      - Capsules: ['startTime', 'endTime', 'duration']
    - Cursors:
      - Values: show
  - Color:
    - TrendItemsColor:
      - Name: Area A_Temperature, Color: #068C45
      - Name: Smooth Temperature, Color: #9D248F
      - Name: Area A_Compressor Power, Color: #CE561B
      - Name: Area C_Temperature, Color: #00A2DD
      - Name: Area A_Compressor Stage, Color: #AC4625
      - Name: High Power, Color: #4055A3
      - Name: Condition with Properties, Color: #4B7ABD
      - Name: Temperature Limit, Color: #E1498E
    - Capsules:
      - Color capsules by: capsuleProperty

Similarly, you can access the color configuration of the trend toolbar by accessing the `color` attribute on the `trend_toolbar` object. 

You can make changes by setting the respective properties.

In [15]:
# Change color of an item on the trend view
item_id = worksheet.trend_toolbar.color.trend_items.items['ID'][0]
worksheet.trend_toolbar.color.trend_items.set_item_color(item_id, '#ffff30')

# Change condition color mode, valid options ('item', 'capsuleProperty', 'capsulePropertyGradient')
worksheet.trend_toolbar.color.capsules.color_capsules_by = 'item'

# Set capsule property, this is the property to be used when setting the capsule property colors or gradient
worksheet.trend_toolbar.color.capsules.capsule_property = 'Running'

# Change colors of current capsule property values, this will automatically change the color mode to 'capsuleProperty'
worksheet.trend_toolbar.color.capsules.capsule_property_color = {'TRANSITION': '#ff3030', 'OFF': '#4B7ABD', 'STAGE 1': '#AC4625', 'STAGE 2': '#D3AA4C'}

# Change capsule property gradient color, this will automatically change the color mode to 'capsulePropertyGradient'
worksheet.trend_toolbar.color.capsules.capsule_property_gradient = {'From': '#ffff30', 'To': '#ff3030'}

worksheet.trend_toolbar

Trend toolbar configurations:
  - View: Trend
  - Show Gridlines: False
  - Hide Uncertainty: True
  - Dimming: True
  - Labels:
    - Signals:
      - Name: lane
      - Description: off
      - Asset: off
      - Asset Path Levels: 3
      - Line Style: off
      - Unit of Measure: axis
      - Custom: lane
      - Custom Labels: ['Model TMP-3100Chill', 'Model CP-6400Delta']
    - Conditions:
      - Name: lane
      - Capsules: ['startTime', 'endTime', 'duration']
    - Cursors:
      - Values: show
  - Color:
    - TrendItemsColor:
      - Name: Area A_Temperature, Color: #ffff30
      - Name: Smooth Temperature, Color: #9D248F
      - Name: Area A_Compressor Power, Color: #CE561B
      - Name: Area C_Temperature, Color: #00A2DD
      - Name: Area A_Compressor Stage, Color: #AC4625
      - Name: High Power, Color: #4055A3
      - Name: Condition with Properties, Color: #4B7ABD
      - Name: Temperature Limit, Color: #E1498E
    - Capsules:
      - Color capsules by: capsuleProperty

Now push to the server and click the link to observe:

In [16]:
spy.workbooks.push(workbooks, path='SPy Documentation Examples >> My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,85,00:00:16.22,0,Success,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,link
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,00:00:02.91,0,Success,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,link


,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,85,0:00:16.220912,0,Success,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,http://localhost:34216/0EFF2C6D-1E8D-EC40-84C8...
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,0:00:02.913101,0,Success,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,http://localhost:34216/0EFF2C6D-1E8D-EC40-84C8...


### Table View Configurations

You can configure the condition table view by accessing the `condition_table_toolbar` attribute on the worksheet object.

You can make changes by setting and calling the respective properties and methods.

In [16]:
workbooks = spy.workbooks.load('Support Files/Example Export.zip')

worksheet = workbooks['Example Analysis'].worksheets['Condition Table Toolbar']

# Ensure condition table view is active rather than simple table view
worksheet.condition_table_toolbar.table_mode = 'condition'

# Enable Transpose
worksheet.condition_table_toolbar.transpose = True

# Enable the Unit of Measure row
worksheet.condition_table_toolbar.add_row_unit_of_measure()

# Enable grouping. The columns that should be used for grouping is specified by setting the condition_table_toolbar.columns.grouped_columns property
worksheet.condition_table_toolbar.group = True

worksheet.condition_table_toolbar

Condition Table Configuration:
  - Table Mode: condition
  - Transpose: True
  - Striped: False
  - Group: True
  - Column Labels: True
  - Units of Measure Row: True
  - Columns:
      - Batch ID:
        - Key: Batch ID
        - Type: Property
        - Grouped: False

You can configure the active columns and their styling using the `column` attribute on `condition_table_toolbar`. In this case, there is only one column currently present in the table, which shows the value of the capsule property 'Batch ID' for each capsule in the table.

In [17]:
# Add a new capsule property column 'Operation' and change it's header to Batch Operation
operation_column = worksheet.condition_table_toolbar.columns.add_column_property('Operation')
operation_column.header = 'Batch Operation'

# Group by the operation column
worksheet.condition_table_toolbar.columns.grouped_columns = [operation_column]

# Add a new signal statistic column for signal 'Solution Concentration' already in the details pane with statistic 'maximum'
solution_conc_signal = worksheet.display_items[worksheet.display_items['Name'] == 'Solution Concentration']
solution_conc_column = worksheet.condition_table_toolbar.columns.add_column_statistic(solution_conc_signal, 'maximum')
solution_conc_column.header = 'Maximum Solution Concentration'
# Turn on column aggregation in order to see the average of the maximum solution concentration across the different operations
solution_conc_column.aggregation_function = 'avg'

worksheet.condition_table_toolbar

Condition Table Configuration:
  - Table Mode: condition
  - Transpose: True
  - Striped: False
  - Group: True
  - Column Labels: True
  - Units of Measure Row: True
  - Columns:
      - Batch ID:
        - Key: Batch ID
        - Type: Property
        - Grouped: False

      - Operation:
        - Key: Operation
        - Type: Property
        - Header: Batch Operation
        - Grouped: True
        - Group Order: 0

      - Solution Concentration maximum:
        - Key: statistics.maximum_0EFF4912-CC09-EAB0-8578-F64D5FFE3EDF
        - Type: Signal Statistic
        - Header: Maximum Solution Concentration
        - Item ID: 0EFF4912-CC09-EAB0-8578-F64D5FFE3EDF
        - Statistic: maximum
        - Grouped: False
        - Aggregation Function: avg

In [18]:
spy.workbooks.push(workbooks, path='SPy Documentation Examples >> My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,83,00:00:01.14,0,Success,0F02459C-3E96-EEB0-A519-A700EDF0238A,link
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,5,00:00:00.31,0,Success,0F02459C-47BE-EAB0-AF40-1CD096A9116C,link


,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,83,0:00:01.143483,0,Success,0F02459C-3E96-EEB0-A519-A700EDF0238A,http://localhost:34216/0F02459C-3E6D-66A0-A922...
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,5,0:00:00.305444,0,Success,0F02459C-47BE-EAB0-AF40-1CD096A9116C,http://localhost:34216/0F02459C-3E6D-66A0-A922...


## Re-importing and Labels

If you **push** a set of workbooks more than once, then by default you will simply overwrite the existing workbooks with the saved content. This can be useful when you are "backing up" content to disk, perhaps for the purposes of version control.

You can choose to **push** and supply a _label_, which will create a separate copy of all of the imported items instead of modifying the existing ones. This is useful when you want to import something that you are iterating on prior to affecting the "published" version. For example, let's push our workbooks with the label of `In Development`:

In [17]:
workbooks = spy.workbooks.load('Support Files/Example Export.zip')

spy.workbooks.push(workbooks, path='SPy Documentation Examples >> My Development Folder', label='In Development')

,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,60,00:00:08.77,0,Success,0EFEEC89-4EF8-6610-A622-81CF23F6326F,link
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,00:00:04.09,0,Success,0EFEEC89-7B95-6470-9192-EDDD5B9109F2,link


,ID,Name,Type,Workbook Type,Count,Time,Errors,Result,Pushed Workbook ID,URL
0,0EFEEC61-3067-73E0-AD1A-BF8C12ECD440,Example Analysis,Workbook,Analysis,60,0:00:08.773897,0,Success,0EFEEC89-4EF8-6610-A622-81CF23F6326F,http://localhost:34216/0EFEEC89-4E5C-6210-8F88...
1,0EFEEC62-575D-77F0-8A42-A7AFA31B0E7B,Example Topic,Workbook,Topic,5,0:00:04.086825,0,Success,0EFEEC89-7B95-6470-9192-EDDD5B9109F2,http://localhost:34216/0EFEEC89-4E5C-6210-8F88...


If you refresh Seeq Workbench, you'll notice that there is now a _My Development Folder_ and a separate copy of the Topic and Analysis that is independent of the original -- including all calculated items.

Pushing with the same value for the `label` argument will overwrite the content for that label. Change the label again if you want yet another separate copy.

## Mapping Items

Sometimes you will have workbooks that are built upon a set of items (signals, conditions, scalars etc) and you would like to programmatically replace that set of items with a different set. For example, perhaps some of your workbooks are built using tags from an on-premise plant historian and you'd like to to switch to using the enterprise historian in the cloud.

In order to achieve this, you must create a _Datasource Map Override_ folder, populate it with modified Datasource Map files, and then reference it via the `spy.workbooks.push(datasource_map_folder=<folder>)` argument.

When you pull a set of workbooks and save them to disk, there will be a series of files in the save location that start with `Datasource_Map_`, one for each datasource that (any of) the workbooks touch. If you want to "re-route" any of the items in those datasources, copy the appropriate datasource map files to a new folder location and then edit them with a text editor. Note that Seeq Data Lab includes an editor:

- In Advanced Mode, right-click on a file and select **Open With > Editor**.
- In Non-Advanced Mode, click the checkbox to the left of the file and select **Edit** from the top set of action buttons.

### Tweaking the Datasource Map

Each Datasource Map file is in the JSON format. Here's what the `Datasource_Map_Time Series CSV Files_Example Data_Example Data` looks like:

```
{
    "Datasource Class": "Time Series CSV Files",
    "Datasource ID": "Example Data",
    "Datasource Name": "Example Data",
    "Item-Level Map Files": [],
    "RegEx-Based Maps": [
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "(?<data_id>.*)"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "${data_id}"
            }
        }
    ]
}
```

You will generally start by focusing on the `RegEx-Based Maps` section. This is a list of _mapping directives_, each with an `Old` and a `New` dictionary. Each key in the dictionary is an item property. In the `Old` dictionary, the value for each key is a Regular Expression that is used to match on the item property's value. When `spy.workbooks.push()` is operating, it runs every item for this datasource through each of the `Old` sections to determine if they match.

Let's take the value of `Type` for the map above: `(?<type>.*)`. Using [regex101.com](https://regex101.com) as an explanatory tool, we can copy and paste `(?<type>.*)` into the Regular Expression text box at the top of its user interface and it will explain what is happening on the far right:

- A _Named Capture Group_ called `type` is being defined.
- The `.*` matches on any character, any number of times.

This effectively means that the entire value of the `Type` property is being assigned to the `type` _Named Capture Group_ (for use later, in the `New` section). The same thing is being done for `Data ID`. However, `Datasource Class` and `Datasource Name` are being explicitly matched to `Time Series CSV Files` and `Example Data`, respectively, and they are not being assigned to a capture group.

Now let's focus on the `New` dictionary. The values in this dictionary are used to "look up" an item and use it in place of the item identified by the `Old` criteria. The default mapping is simple: It just finds items that are a 100% match of the `Type` and `Data ID` properties. (You can look at Item Properties for any item in Seeq Workbench by clicking the (i) button to the left of the item in the _Details_ pane.) If you are pulling and pushing within the same server without altering the map, this conveniently means that the old and new items are the same, and nothing changes.

But what if you need to map to a different datasource and you want to use the `Name` property in a somewhat complex way? Let's look at a possible example:

```
{
    "Datasource Class": "OSIsoft PI",
    "Datasource ID": "442A2678-A76F-49C4-A702-4CC494701D6C",
    "Datasource Name": "PIPRDSRV012",
    "Item-Level Map Files": [],
    "RegEx-Based Maps": [
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "OSIsoft PI",
                "Datasource Name": "PIPRDSRV012",
                "Name": "(?<name>.*)"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "ADX",
                "Datasource Name": "PRDDATALAKE",
                "Name": "PIPRDSRV012_${name}"
            }
        }
    ]
}
```

Here we are mapping from an OSIsoft PI datasource (`PIPRDSRV012`) to a Microsoft ADX data lake (`PRDDATALAKE`), and we are using the `Name` property for mapping. When we are looking up the equivalent tag in the data lake, we are prepending `PIPRDSRV012_` to the name, because that is the (fictional) naming convention in this hypothetical data lake.

In this way, you can specify a relatively complex set of rules for mapping. You can have as many Old/New dictionaries in the `RegEx-Based Maps` list as you want, and SPy will match against the `Old` dictionary in the order you specify them. You can only use the following properties in the `New` dictionary: `Datasource Class`, `Datasource ID`, `Datasource Name`, `Data ID`, `Type`, `Name`, `Description`, `Username`, `Path`, `Asset`.


## Importing to a Different Seeq Server

You may wish to copy content to a new/different Seeq Server by exporting and then importing. For example, you might have a _development_ server where you iterate on content and a _production_ server that you publish to when finished.

In order to accomplish this, you'll do one of two actions:

- If you're using the SPy module within Seeq Data Lab, you'll copy the exported folder to the other version of Seeq Data Lab and then push it from there.
- If you're using the SPy module with your own Python set up, you'll log in to the other server and push it.

The default Datasource Map files that are part of the export may need to be tweaked if the destination server has differently-named datasources. See _Custom Datasource Mapping_ above.

## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [18]:
help(spy.workbooks.search)

Help on function search in module seeq.spy.workbooks._search:

search(query, *, content_filter='owner', all_properties=False, recursive=False, include_archived=False, errors=None, quiet=None, status=None, session: 'Optional[Session]' = None)
    Issues a query to the Seeq Server to retrieve metadata for workbooks.
    This metadata can be used to pull workbook definitions into memory.
    
    Parameters
    ----------
    query : dict
        A mapping of property / match-criteria pairs. Match criteria uses
        the same syntax as the Data tab in Seeq (contains, or glob, or regex).
        Available options are:
    
        =================== ===================================================
        Property            Description
        =================== ===================================================
        ID                  ID of the workbook, as seen in the URL.
        Name                Name of the workbook.
        Path                Path to the workbook thro

In [19]:
help(spy.workbooks.pull)

Help on function pull in module seeq.spy.workbooks._pull:

pull(workbooks_df: 'Union[pd.DataFrame, str]', *, include_referenced_workbooks: 'bool' = True, include_inventory: 'bool' = True, include_annotations: 'bool' = True, include_images: 'bool' = True, include_rendered_content: 'bool' = False, include_access_control: 'bool' = True, include_archived: 'bool' = None, specific_worksheet_ids: 'Optional[List[str]]' = None, as_template_with_label: 'str' = None, errors: 'Optional[str]' = None, quiet: 'Optional[bool]' = None, status: 'Optional[Status]' = None, session: 'Optional[Session]' = None) -> 'Optional[WorkbookList]'
    Pulls the definitions for each workbook specified by workbooks_df into
    memory as a list of Workbook items.
    
    Parameters
    ----------
    workbooks_df : {str, pd.DataFrame}
        A DataFrame containing 'ID', 'Type' and 'Workbook Type' columns that
        can be used to identify the workbooks to pull. This is usually created
        via a call to spy.work

In [20]:
help(spy.workbooks.save)

Help on function save in module seeq.spy.workbooks._save:

save(workbooks, folder_or_zipfile: 'Optional[str]' = None, *, datasource_map_folder: 'Optional[str]' = None, include_rendered_content: 'bool' = False, pretty_print_html=False, overwrite: 'bool' = False, errors: 'Optional[str]' = None, quiet: 'Optional[bool]' = None, status: 'Optional[Status]' = None)
    Saves a list of workbooks to a folder on disk from Workbook objects in
    memory.
    
    Parameters
    ----------
    workbooks : {Workbook, list[Workbook]}
        A Workbook object or list of Workbook objects to save.
    
    folder_or_zipfile : str, default os.getcwd()
        A folder or zip file on disk to which to save the workbooks. It will
        be saved as a "flat" set of subfolders, no other hierarchy will be
        created. The string must end in ".zip" to cause a zip file to be
        created instead of a folder.
    
    datasource_map_folder : str, default None
        Specifies a curated set of datasourc

In [21]:
help(spy.workbooks.load)

Help on function load in module seeq.spy.workbooks._load:

load(folder_or_zipfile, *, as_template_with_label: 'str' = None, errors: 'Optional[str]' = None, quiet: 'Optional[bool]' = None, status: 'Optional[Status]' = None) -> 'WorkbookList'
    Loads a list of workbooks from a folder on disk into Workbook objects in
    memory.
    
    Parameters
    ----------
    folder_or_zipfile : str
        A folder or zip file on disk containing workbooks to be loaded. Note
        that any subfolder structure will work -- this function will scan for
        any subfolders that contain a Workbook.json file and assume they should
        be loaded.
    
    as_template_with_label : str
        Causes the workbooks to be loaded as templates (either AnalysisTemplate
        or TopicTemplate) with the label specified. See the Workbook Templates
        documentation notebook for more information about templates.
    
    errors : {'raise', 'catalog'}, default 'raise'
        If 'raise', any errors 

In [22]:
help(spy.workbooks.push)

Help on function push in module seeq.spy.workbooks._push:

push(workbooks, *, path: 'Optional[str]' = None, owner: 'Optional[str]' = None, label: 'Optional[str]' = None, datasource: 'Optional[str]' = None, datasource_map_folder: 'Optional[str]' = None, use_full_path: 'bool' = False, access_control: 'Optional[str]' = None, override_max_interp: 'bool' = False, include_inventory: 'Optional[bool]' = None, include_annotations: 'bool' = True, refresh: 'bool' = True, lookup_df: 'pd.DataFrame' = None, specific_worksheet_ids: 'Optional[List[str]]' = None, create_dummy_items_in_workbook: 'Optional[str]' = None, assume_dependencies_exist: 'bool' = False, reconcile_inventory_by: 'str' = 'id', global_inventory: 'Optional[str]' = None, item_map: 'Optional[ItemMap]' = None, errors: 'Optional[str]' = None, quiet: 'Optional[bool]' = None, status: 'Optional[Status]' = None, session: 'Optional[Session]' = None, scope_globals_to_workbook: 'Optional[bool]' = None) -> 'pd.DataFrame'
    Pushes workbooks int